In [ ]:
! pip install langchain

In [ ]:
! pip install langchain-openai

Remeber to set the OPENAI_API_KEY environment variable before running the code.
`os.environ["OPENAI_API_KEY"] = "<>"`

# Building A Language Model Application
### LLMS: Get predictions from a language model

In [1]:
from langchain_openai import OpenAI

In [11]:
llm = OpenAI(temperature=0)

In [14]:
prompt = "What are 5 vacation destinations for someone who likes to eat pasta?"
output = llm.invoke(prompt)


In [15]:
print(output)


1. Italy: As the birthplace of pasta, Italy is a must-visit for any pasta lover. With regions like Tuscany, Emilia-Romagna, and Sicily known for their specific types of pasta dishes, this country offers endless options for delectable pasta meals.

2. New York City, USA: Known as a melting pot of cultures, New York City has a diverse range of Italian restaurants that serve up mouth-watering pasta dishes. From traditional Italian cuisine to modern and creative interpretations of pasta, this city has it all.

3. Buenos Aires, Argentina: Buenos Aires has a strong Italian influence, making it a great destination for pasta lovers. With a variety of homemade pastas and traditional Italian dishes influenced by local flavors like beef and ample use of herbs, this city offers a unique twist on classic pasta dishes.

4. Paris, France: While primarily known for its French cuisine, Paris has a thriving Italian food scene. Along with traditional French dishes, visitors can find delicious handmade p

### Exercise 1

#### Create a prompt and generate a completion using the OpenAI interface of langchain.

### Chat model

In [16]:
from langchain_openai import ChatOpenAI

In [18]:
chat_model = ChatOpenAI(temperature=0, model="gpt-4")

In [19]:
from langchain_core.messages import HumanMessage

In [21]:
text = "What would be a good company name for a dmkk ks"
messages = [
    HumanMessage(content=text)
    ]

output = chat_model.invoke(messages).content
print(output)

"Rainbow Steps"


### Exercise 2

#### Create a prompt and generate a completion using the ChatOpenAI interface of langchain.

### Prompt Templates: Manage prompts for LLMs

In [23]:
from langchain.prompts import PromptTemplate

In [36]:
prompt = PromptTemplate(
    input_variables=["food", "country"],
    template="What are 5 vacation {country} for someone who likes to eat {food}?",
)

In [38]:
output = prompt.format(food="dessert",country="italy")

In [39]:
print(output)

What are 5 vacation italy for someone who likes to eat dessert?


In [40]:
print(llm.invoke(output))



1. Florence: Known as the birthplace of gelato, Florence offers a plethora of delicious dessert options including traditional gelato flavors like pistachio and stracciatella, as well as tasty pastries like the famous Florentine almond biscotti.

2. Naples: This southern Italian city is famous for its Neapolitan pizza, but it also has a rich history of dessert-making. Try the signature sfogliatelle, a flaky pastry filled with ricotta cheese and candied fruit, or indulge in a traditional zeppole, a deep-fried pastry topped with powdered sugar.

3. Sicily: As the largest island in the Mediterranean, Sicily has a diverse culinary scene that includes a wide range of desserts. Don't miss out on trying cannoli, a sweet pastry filled with ricotta cream and chocolate chips, or marzipan fruits, a traditional Sicilian treat made from almond paste.

4. Rome: In addition to its famous gelato shops, Rome is also known for its rich and decadent desserts. Take a stroll through the historic streets a

### Exercise 3

#### Create a prompt and generate a completion using the PromptTemplates, BE CREATIVE :D

### Same for chat

In [41]:
from langchain.prompts.chat import ChatPromptTemplate


In [42]:
template = "You are a helpful assistant that translates {input_language} to {output_language}."
human_template = "{text}"

In [43]:

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template),
])


In [44]:
prompt = chat_prompt.format_messages(input_language="English", output_language="French", text="I love programming.")

In [45]:
print(prompt)

[SystemMessage(content='You are a helpful assistant that translates English to French.'), HumanMessage(content='I love programming.')]


In [46]:
chat_model.invoke(prompt).content

"J'aime la programmation."

## Output parsers

OutputParsers convert the raw output of a language model into a format that can be used downstream. There are a few main types of OutputParsers, including:

- Convert text from LLM into structured information (e.g. JSON)
- Convert a ChatMessage into just a string
- Convert the extra information returned from a call besides the message (like OpenAI function invocation) into a string.

In [47]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [48]:
output_parser = CommaSeparatedListOutputParser()

In [50]:
output = output_parser.parse("hi, how, are, you?")

In [51]:
print(output)
for word in output:
    print(word)

['hi', 'how', 'are', 'you?']
hi
how
are
you?


Apart from the parse method, each output parse comes with some instructions that should be added to the prompt to guide the LLM on how to generate the output.

In [52]:
output_parser.get_format_instructions()

'Your response should be a list of comma separated values, eg: `foo, bar, baz`'

In [53]:
# Let's see how to use the output parser with a chat model
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that helps finding places to visits based on a location\n\n{format_instructions}"),
        ("human", "{text}"),
    ]
    )
formatted_prompt = prompt.format_prompt(text="Paris", format_instructions=output_parser.get_format_instructions())  

In [54]:
print(formatted_prompt)

messages=[SystemMessage(content='You are a helpful assistant that helps finding places to visits based on a location\n\nYour response should be a list of comma separated values, eg: `foo, bar, baz`'), HumanMessage(content='Paris')]


In [57]:
output = chat_model.invoke(formatted_prompt).content

In [58]:
output_parser.parse(output)

['Eiffel Tower',
 'Louvre Museum',
 'Notre-Dame Cathedral',
 'Montmartre',
 'Palace of Versailles',
 'Sainte-Chapelle',
 'Seine River',
 "Musée d'Orsay",
 'Champs-Élysées',
 'Sacré-Cœur Basilica']

Let's see other parsers...

#### JSON parser

Very useful for structured information. It will convert the raw output of the LLM into a JSON object.

In [25]:
from typing import List
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field

In [26]:
class Recipe(BaseModel):
    name: str = Field(description="The name of the recipe.")
    difficulty: int = Field(description="The difficulty level of the recipe from 1 to 10.")
    time: int = Field(description="The time needed to make the recipe in minutes.")
    ingredients: List[str] = Field(description="The list of ingredients needed for the recipe.")
    steps: List[str] = Field(description="The list of steps to make the recipe.")

In [27]:
output_parser = JsonOutputParser(pydantic_object=Recipe)

# let's see how the format instructions look like
output_parser.get_format_instructions()

'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"name": {"title": "Name", "description": "The name of the recipe.", "type": "string"}, "difficulty": {"title": "Difficulty", "description": "The difficulty level of the recipe from 1 to 10.", "type": "integer"}, "time": {"title": "Time", "description": "The time needed to make the recipe in minutes.", "type": "integer"}, "ingredients": {"title": "Ingredients", "description": "The list of ingredients needed for the recipe.", "type": "array", "items": {"type": "string"}}, "steps": {"title": "Steps", "description": "The li

### Exercise 4

#### Put a JSON parser in the prompt and generate a completion using the OpenAI interface of langchain.

In [28]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant that provides recipes based on a query\n\n{format_instructions}"),
        ("human", "{text}"),
    ])
query = "What a easy recipes for pasta?"
json_result = output_parser.parse(
    llm.invoke(
        prompt.format_messages(text=query, format_instructions=output_parser.get_format_instructions())
        )
    )

In [30]:
print(type(json_result))
json_result['steps']

<class 'dict'>


['Cook pasta according to package instructions.',
 'Heat pasta sauce in a separate pan.',
 'Drain pasta and mix with sauce.',
 'Sprinkle parmesan cheese on top and serve.']

The result is a python dict that can be used downstream.

### Chains: Combine LLMs and prompts in multi-step workflows

The power of langchain comes from the ability to chain together multiple LLMs and prompts to create complex workflows.

It also comes with a few built-in chains that can be used to create common workflows.

In [59]:
prompt = prompt.partial(format_instructions=output_parser.get_format_instructions())
chain = prompt | llm | output_parser

In [60]:
chain.invoke({"text": "What is a recipe with eggs and rice?"})

['Egg Fried Rice', 'Spanish Omelette', 'Egg Fried Cauliflower Rice']

#### Example extraction chain

In [34]:
from langchain.chains.openai_functions.extraction import create_extraction_chain_pydantic

chain = create_extraction_chain_pydantic(
    pydantic_schema=Recipe,
    llm=chat_model
)
# read text from txt file
recipe_text = open("carbonara.txt", "r").read()

In [37]:
extracted_result = chain.invoke(recipe_text)

We have our structured output from a text without implementing anything!

In [36]:
extracted_class = extracted_result['text'][0]
print("Recipe name:", extracted_class.name)
print("Difficulty:", extracted_class.difficulty)
print("Time:", extracted_class.time)
print("Ingredients:", extracted_class.ingredients)

Recipe name: SPAGHETTI ALLA CARBONARA
Difficulty: 2
Time: 20
Ingredients: ['Pancetta or bacon', 'Olive oil', 'Butter', 'Garlic cloves, peeled and lightly crushed', 'White wine, dry', 'Salt', 'Thin spaghetti', 'Eggs', 'Peccorino romano cheese, freshly grated', 'Parmesan cheese, freshly grated', 'Black pepper, freshly ground', 'Parsley, chopped fine']


### Agents: Dynamically call chains based on user input

In [ ]:
! pip install -U langchain-community tavily-python langchainhub

#### Tools

In [62]:
from langchain.agents import load_tools
tools = load_tools(["serpapi","llm-math"], llm=llm)

#### Finally, let's initialize an agent with:
1. The tools
2. The language model
3. The type of agent we want to use.
4. The prompt that guides the agent

In [63]:
from langchain import hub
from langchain.agents import create_react_agent

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

agent = create_react_agent(chat_model, tools, prompt)

See list of agents types [here](https://python.langchain.com/docs/modules/agents/agent_types/)

Finally, we combine the agent (the brains) with the tools inside the AgentExecutor (which will repeatedly call the agent and execute tools).

In [64]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [66]:
# Now let's test it out!
agent_executor.invoke({"input":"What is the latest episode of One Piece?"})



> Entering new AgentExecutor chain...
I need to find the most recent episode of One Piece, which is a current event. I should use the search engine to find this information.
Action: Search
Action Input: Latest episode of One Piece['CP-0 led by Rob Lucci makes a forced landing at Egghead.', 'Preparations for Interception! Rob Lucci Strikes! type: One Piece: Season 21, Episode 11.', 'Preparations for Interception! Rob Lucci Strikes! entity_type: tv, tvm.', 'Preparations for Interception! Rob Lucci Strikes! main_tab_text: Overview.', 'Preparations for Interception! Rob Lucci Strikes! kgmid: /g/11vwjqlx73.', 'Preparations for Interception! Rob Lucci Strikes! show: One Piece.', 'One Piece is an anime television series based on the manga series of the same name. As of 2024 it has more than 1,000 episodes.', 'Stream and watch the anime One Piece on Crunchyroll. Monkey. D. Luffy refuses to let anyone or anything stand in the way of his quest to become the king of ...', 'When will One Piece e

{'input': 'What is the latest episode of One Piece?',
 'output': 'The latest episode of One Piece is "Preparations for Interception! Rob Lucci Strikes!" from Season 21, Episode 11.'}